In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
import random
from selenium.webdriver.chrome.options import Options
import os
import subprocess
import json

chrome_options = Options()
filename = "data"
link = "https://www.eventbrite.com/d/india--mumbai/all-events/?page="

# chrome_options.add_argument("--headless")
browser = webdriver.Chrome(options=chrome_options)
record = []
e = []
le = 0

event_ids = []

def Selenium_extractor():
    
    action = ActionChains(browser)
    # with open("eventbrite_data/ev.html", "w") as file:
    #     file.write(browser.page_source)
    time.sleep(2)

    a = browser.find_elements(By.XPATH, '//div[@data-testid="event-card-tracking-layer"]')
    
    for i in a:
        event_ids.append(i.get_attribute("data-event-id"))
        

for i in range(7):
    time.sleep(2)
    try:
        browser.get(str(link) + str(i+1))
        time.sleep(2)
        Selenium_extractor()
    except:
        continue
browser.quit()
unique_event_ids = [*set(event_ids)]

In [19]:
website1 = "https://www.eventbrite.com/api/v3/destination/events/?event_ids="
website2 = "&expand=event_sales_status,image,primary_venue,saves,ticket_availability,primary_organizer,public_collections"
for i in unique_event_ids:
    cmd = subprocess.Popen(["GET", website1 + str(i) + website2], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(2)
    output, error = (cmd.communicate())
    time.sleep(2)
    with open("json_data/event" + str(i) + ".json", "w") as file:
        file.write((output.decode('utf-8')))

In [23]:
import json
df = pd.DataFrame({"Name" : [], "image_link" : [], "tickets_url" : [], "start_date" : [],"start_time" : [],  "end_date" : [], "end_time" : [],"venue" : [], "address_1" : [], "address_2" : [], "organizer" : [], "organizer_url" : [],"url" : []})
for i in unique_event_ids:
    with open("json_data/event" + str(i) + ".json") as data_file:
        # print(type(data_file))
        parsed_data = json.load(data_file)
        # print(type(parsed_data))
        # print(type(json.load(data_file)))
    event = parsed_data['events'][0]
    event_data = []
    try:
        event_data.append(event['name'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['image']['url'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['tickets_url'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['start_date'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['start_time'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['end_date'])
    except:
        event_data.append("Unavailable")
    try:    
        event_data.append(event['end_time'])
    except:
        event_data.append("Unavailable")
    try:    
        event_data.append(event['primary_venue']['name'])
    except:
        event_data.append("Unavailable")
    try:    
        event_data.append(event['primary_venue']['address']['address_1'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['primary_venue']['address']['address_2'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['primary_organizer']['name'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['primary_organizer']['url'])
    except:
        event_data.append("Unavailable")
    try:
        event_data.append(event['url'])
    except:
        event_data.append("Unavailable")
    
    df.loc[len(df)] = event_data

output_path='data_scrapped/eventbrite_data.csv'
df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))